In [60]:
import requests
from bs4 import BeautifulSoup
import json
import csv
from datetime import datetime

In [61]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [62]:
# Đường dẫn đến ChromeDriver
driver_path = 'path/to/chromedriver'

In [63]:
# Khởi động ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Truy cập trang web
url = 'https://www.imdb.com/search/title/?title=axel&title_type=feature&release_date=2022-01-01,2024-07-10&sort=year,desc'
driver.get(url)
# Đợi một chút để trang web tải xong
wait = WebDriverWait(driver, 10)

In [64]:
# Hàm để cuộn trang và click vào nút "More"
def load_more_content(driver):
    while True:
        try:
            # Cuộn xuống cuối trang
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Đợi một chút để nội dung mới tải xong
            
            # Đợi cho đến khi nút "More" có thể click được
            more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'ipc-see-more__text')))
            more_button.click()
            time.sleep(2)  # Đợi một chút để nội dung mới tải xong
        except Exception as e:
            print(f'Error: {e}')
            break

In [65]:
# Gọi hàm để cuộn trang và click vào nút "More"
load_more_content(driver)

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x002CC203+27395]
	(No symbol) [0x00263E04]
	(No symbol) [0x00161B7F]
	(No symbol) [0x001A2C65]
	(No symbol) [0x001A2D3B]
	(No symbol) [0x001DEC82]
	(No symbol) [0x001C39E4]
	(No symbol) [0x001DCB24]
	(No symbol) [0x001C3736]
	(No symbol) [0x00197541]
	(No symbol) [0x001980BD]
	GetHandleVerifier [0x00583AB3+2876339]
	GetHandleVerifier [0x005D7F7D+3221629]
	GetHandleVerifier [0x0034D674+556916]
	GetHandleVerifier [0x0035478C+585868]
	(No symbol) [0x0026CE44]
	(No symbol) [0x00269858]
	(No symbol) [0x002699F7]
	(No symbol) [0x0025BF4E]
	BaseThreadInitThunk [0x776BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77A580CE+286]
	RtlGetAppContainerNamedObjectPath [0x77A5809E+238]
	(No symbol) [0x00000000]



In [66]:
# lấy data cua toan bo cac bo phim trong trang web
movie_elements = driver.find_elements(By.CSS_SELECTOR, '.ipc-metadata-list.ipc-metadata-list--dividers-between.sc-748571c8-0.jmWPOZ.detailed-list-view.ipc-metadata-list--base')
# Extract HTML nội dung từ mỗi element
html_content = []
for element in movie_elements:
    html_content.append(element.get_attribute('outerHTML'))

# Join all HTML content into a single string
html_str = ''.join(html_content)


In [67]:
# Đóng trình duyệt
driver.quit()

In [68]:
# Parse HTML with BeautifulSoup
soup = BeautifulSoup(html_str, 'html.parser')
link_film_acc = []
divs = soup.find_all('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b189961a-9 iALATN dli-title')
cout = 1
# Lặp qua từng thẻ <div> để lấy tên phim và liên kết
for div in divs:
    # Tìm thẻ <a> trong thẻ <div>
    link = div.find('a', class_='ipc-title-link-wrapper')
    
    # Kiểm tra nếu tìm thấy thẻ <a> và lấy href từ thuộc tính 'href'
    if link:
        link_film = link.get('href')
        link_film_acc.append(link_film)
        cout += 1

In [69]:
# Gui yeu cau với User-Agent cua trinh duyet that
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [82]:
# Truy cập trang web

for i in link_film_acc:
    url_film = f'https://www.imdb.com{i}'
    r_content = requests.get(url_film, headers=headers)
    soup_detail = BeautifulSoup(r_content.content, 'html.parser')
    # Title cua bo phim
    title_film_detail = soup_detail.find('span', class_='hero__primary-text')
    # Noi dung cua Detail
    detail_release_date = soup_detail.find('li', {'data-testid': 'title-details-releasedate'})
    # release_date_first
    release_date_html = detail_release_date.find('a', class_='ipc-metadata-list-item__list-content-item')
    release_date_str_full = release_date_html.text
    release_date_str = release_date_str_full.split('(')[0].strip() 
    release_date_first = datetime.strptime(release_date_str, "%B %d, %Y")
    # Country of origin
    detail_country_origin = soup_detail.find('li', {'data-testid': 'title-details-origin'})
    country_origin = detail_country_origin.find('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    # IMDB Rating
    detail_rating = soup_detail.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__score'})
    rating_str = detail_rating.find('span',class_='sc-eb51e184-1 cxhhrI') if detail_rating else None
    rating = rating_str.text.strip() if rating_str else None
    # Budget
    detail_budget = soup_detail.find('li', {'data-testid': 'title-boxoffice-budget'})
    budget_full = detail_budget.find('span',class_='ipc-metadata-list-item__list-content-item') if detail_budget else None
    budget_str = budget_full.text
    budget = budget_str.split('(')[0].strip() 
    # Genres
    detail_genres = soup_detail.find('div', {'data-testid': 'genres'})
    genres = detail_genres.find('span',class_='ipc-chip__text') 
    print(url_film, title_film_detail.text, release_date_first, country_origin.text, rating, budget, genres.text)

https://www.imdb.com/title/tt3083016/?ref_=sr_t_1 Cảnh Sát Beverly Hills: Axel F 2024-07-03 00:00:00 United States 6.5 $150,000,000 Action
https://www.imdb.com/title/tt14257104/?ref_=sr_t_2 Axel Falcon 2024-06-01 00:00:00 United Kingdom None £4,000 Action


In [ ]:
print()